## Sample study for forecasting Daily Cash Flow in branches of 3 zones of PNB, a Public Sector Bank: Agra, Chandigarh and Gurugram
### 10 branches of Agra zone and 12 each from Chandigarh and Gurugram are identified as sample.
##### 1. Here we have imported the libraries at first.
##### 2. The data is read and preprocessed to make it in the desired form.
##### 3. A function is made which is having the following parts:
1. 4 dataframes are defined : two for Pay(test & train) and two for Receipt(test & train)
2. A for-loop is run taking each branch at a time. Inside the for-loop:
    1. A holiday function is defined which takes the dataframe as an input and forms a holiday dataframe with the list of dates having holiday effects with the name of the holiday, i.e., weekends, national and local holidays, and another two columns viz. lower_window and upper_window.
    2. The initial dataframe is divided into two dataframes, one for Pay and one for Receipt. These dataframes are futher divided into test and train sets.
    3. A forecast function is defined which takes the pay/receipt train/test dataset, the future dates to be forecasted as an input and returns a dataframe having all the forecasted values and its confidence intervals and the holiday effects.
    4. Two functions, one for rmse, and one for mape, are defined which are used in the details function, which is taking the data and the forecasted data as input and returns some specified metrics from the input.
    5. All the details are appended in a dataframe. Thus we get 4 dataframes: pay_train, pay_test, receipt_train, receipt_test. These 4 dataframes are saved in the system directly.

In [1]:
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import time

C:\Users\RAKTIMA\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
datafr1 = pd.read_csv("manjis_zones_3_shared.csv")
datafr1.columns
datafr1 = datafr1.drop(['Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Agra Zone Branches', 'Chandigarh Zone Branches',
       'Gurugram Zone Branches'], axis = 1)
datafr1.columns = ['Branch', 'Date', 'TOT_RCPT', 'TOT_PAY']
datafr1['Date'] = pd.to_datetime(datafr1['Date'], format = '%d-%m-%Y', infer_datetime_format = True)
datafr1

Branch       Date  TOT_RCPT    TOT_PAY
0      Agr_Br_1 2021-01-01  39.26000  28.978000
1      Agr_Br_1 2021-01-02  28.56600  35.458220
2      Agr_Br_1 2021-01-04  44.89700  29.639000
3      Agr_Br_1 2021-01-05  50.44600  30.010601
4      Agr_Br_1 2021-01-06  39.28800  25.266000
...         ...        ...       ...        ...
13484  Gur_Br_9 2022-06-24  16.36322  22.599000
13485  Gur_Br_9 2022-06-27   7.24390  18.126000
13486  Gur_Br_9 2022-06-28  29.39360  27.473000
13487  Gur_Br_9 2022-06-29  11.93360  25.846000
13488  Gur_Br_9 2022-06-30   9.47400  10.989000

[13489 rows x 4 columns]

In [3]:
datafr1['Branch'].unique()

array(['Agr_Br_1', 'Agr_Br_10', 'Agr_Br_2', 'Agr_Br_3', 'Agr_Br_4',
       'Agr_Br_5', 'Agr_Br_6', 'Agr_Br_7', 'Agr_Br_8', 'Agr_Br_9',
       'Chan_Br_1', 'Chan_Br_10', 'Chan_Br_11', 'Chan_Br_12', 'Chan_Br_2',
       'Chan_Br_3', 'Chan_Br_4', 'Chan_Br_5', 'Chan_Br_6', 'Chan_Br_7',
       'Chan_Br_8', 'Chan_Br_9', 'Gur_Br_1', 'Gur_Br_10', 'Gur_Br_11',
       'Gur_Br_12', 'Gur_Br_2', 'Gur_Br_3', 'Gur_Br_4', 'Gur_Br_5',
       'Gur_Br_6', 'Gur_Br_7', 'Gur_Br_8', 'Gur_Br_9'], dtype=object)

In [4]:
def whole(data):
    global df_pay_train 
    df_pay_train = pd.DataFrame(columns = ['rmse', 'mape', 'avg_act', 'avg_pred', 'stdev', 'min', 'max', 'cnt_neg_pred'])
    global df_pay_test 
    df_pay_test = pd.DataFrame(columns = ['rmse', 'mape', 'avg_act', 'avg_pred', 'stdev', 'min', 'max', 'cnt_neg_pred'])
    global df_receipt_train  
    df_receipt_train = pd.DataFrame(columns = ['rmse', 'mape', 'avg_act', 'avg_pred', 'stdev', 'min', 'max', 'cnt_neg_pred'])
    global df_receipt_test 
    df_receipt_test = pd.DataFrame(columns = ['rmse', 'mape', 'avg_act', 'avg_pred', 'stdev', 'min', 'max', 'cnt_neg_pred'])
    bran = data['Branch'].unique()
    x = 0
    for x in range(len(bran)):
        data12 = bran[x]
        data1 = data[data['Branch'] == data12]
        data1.drop('Branch', inplace = True, axis = 1)
        data1.columns
        data1 = data1.reset_index(drop = True)

        def holiday(data1):
            global hol
            upper_window = []
            lower_window = [0]
            da1 = data1['Date']
    
            for i in range(len(da1) - 1):
                deltap = da1[i+1] - da1[i]
                upper_window.append(deltap.days - 1)
            for i in range(len(da1)):
                if i != 0:
                    deltan = da1[i-1] - da1[i]
                    lower_window.append(deltan.days + 1)
            upper_window.append(0)

            h = pd.concat([da1, pd.DataFrame(lower_window), pd.DataFrame(upper_window)], axis = 1)
            h.columns = ["ds", "lower_window", "upper_window"]

            h2 = h.drop(h[(h['lower_window'] == 0) & (h['upper_window'] == 0)].index)
            h2 = h2.reset_index(drop = True)

            ld = list(h2['ds'])
            lw = list(h2['lower_window'])
            lu = list(h2['upper_window'])

            holiday = []
            for i in range(len(h2)):
                if((ld[i] == pd.to_datetime('2021-01-25')) | (ld[i] == pd.to_datetime('2021-01-27')) | (ld[i] == pd.to_datetime('2021-04-01')) | 
                (ld[i] == pd.to_datetime('2021-04-03')) | (ld[i] == pd.to_datetime('2021-08-14'))| (ld[i] == pd.to_datetime('2021-08-16')) | 
                (ld[i] == pd.to_datetime('2021-10-01')) | (ld[i] == pd.to_datetime('2021-10-03')) | (ld[i] == pd.to_datetime('2021-12-24')) | 
                (ld[i] == pd.to_datetime('2021-12-26'))):
                    holiday.append('national')       

                elif (lw[i] == 0) & (lu[i] == 1):
                    holiday.append('hol_wk_sat1')

                elif (lw[i] == -1) & (lu[i] == 0):
                    holiday.append('hol_wk_mon1')

                elif (lw[i] == 0) & (lu[i] == 2):
                    holiday.append('hol_wk_fri2')

                elif (lw[i] == -2) & (lu[i] == 0):
                    holiday.append('hol_wk_mon2')

                else:
                    holiday.append('local')

            hol = pd.concat([pd.DataFrame(holiday), h2], axis = 1)
            hol.columns = ['holiday', 'ds', 'lower_window', 'upper_window']
            return hol
        holi = holiday(data1)
        
        data1_p = data1.loc[:, ['Date','TOT_PAY']]
        data1_r = data1.loc[:, ['Date', 'TOT_RCPT']]
        data1_p = data1_p[data1_p['TOT_PAY'] > 0.10]
        data1_r = data1_r[data1_r['TOT_RCPT'] > 0.10]
        data1_p.columns = ['ds', 'y']
        data1_r.columns = ['ds', 'y']

        p_train = data1_p.loc[(data1_p['ds'] <= '2022-05-31')]
        p_test = data1_p.loc[(data1_p['ds'] > '2022-05-31')]
        r_train = data1_r.loc[(data1_r['ds'] <= '2022-05-31')]
        r_test = data1_r.loc[(data1_r['ds'] > '2022-05-31')]


        def forecast(data2, future):
            m = Prophet(yearly_seasonality = False, weekly_seasonality = True, daily_seasonality = False,
                       holidays = holi)
            m.fit(data2)
            forecast1 = m.predict(future)
            return forecast1

        pred_p_train = forecast(p_train, pd.DataFrame({'ds' : p_train['ds']}))
        pred_p_test = forecast(p_train, pd.DataFrame({'ds' : p_test['ds']}))
        pred_r_train = forecast(r_train, pd.DataFrame({'ds' : r_train['ds']}))
        pred_r_test = forecast(r_train, pd.DataFrame({'ds' : r_test['ds']}))

        def rmse(actual, pred):
            result = sqrt(mean_squared_error(actual, pred))
            return result

        def mape(actual1, pred1):
            actual1 = np.array(actual1)
            pred1 = np.array(pred1)
            result1 = np.mean(np.abs((actual1 - pred1) / actual1)) * 100
            return result1

        def details(data, fore_data):
            rms_err = rmse(data['y'], fore_data['yhat']) 
            map_err = mape(data['y'], fore_data['yhat'])
            avg_act = data['y'].mean()
            avg_pred = fore_data['yhat'].mean()
            stdev = fore_data['yhat'].std()
            minimum = fore_data['yhat'].min()
            maximum = fore_data['yhat'].max()
            cnt_neg_pred = (fore_data['yhat'] <= 0).sum()
    
            df1 = pd.DataFrame([rms_err, map_err, avg_act, avg_pred, stdev, minimum, maximum, cnt_neg_pred])
            df1 = df1.T
            df1.columns = ['rmse', 'mape', 'avg_act', 'avg_pred', 'stdev', 'min', 'max', 'cnt_neg_pred']
            return df1
        
        dptr = details(p_train, pred_p_train)
        dpte = details(p_test, pred_p_test)
        drtr = details(r_train, pred_r_train)
        drte = details(r_test, pred_r_test)
   
        frames1 = [df_pay_train, dptr]
        frames2 = [df_pay_test, dpte]
        frames3 = [df_receipt_train, drtr]
        frames4 = [df_receipt_test, drte]
        df_pay_train = pd.concat(frames1)
        df_pay_test = pd.concat(frames2)
        df_receipt_train = pd.concat(frames3)
        df_receipt_test = pd.concat(frames4)
    
    df_pay_train.index = ['Agr_Br_1', 'Agr_Br_10', 'Agr_Br_2', 'Agr_Br_3', 'Agr_Br_4',
       'Agr_Br_5', 'Agr_Br_6', 'Agr_Br_7', 'Agr_Br_8', 'Agr_Br_9',
       'Chan_Br_1', 'Chan_Br_10', 'Chan_Br_11', 'Chan_Br_12', 'Chan_Br_2',
       'Chan_Br_3', 'Chan_Br_4', 'Chan_Br_5', 'Chan_Br_6', 'Chan_Br_7',
       'Chan_Br_8', 'Chan_Br_9', 'Gur_Br_1', 'Gur_Br_10', 'Gur_Br_11',
       'Gur_Br_12', 'Gur_Br_2', 'Gur_Br_3', 'Gur_Br_4', 'Gur_Br_5',
       'Gur_Br_6', 'Gur_Br_7', 'Gur_Br_8', 'Gur_Br_9']
    
    df_pay_test.index = ['Agr_Br_1', 'Agr_Br_10', 'Agr_Br_2', 'Agr_Br_3', 'Agr_Br_4',
       'Agr_Br_5', 'Agr_Br_6', 'Agr_Br_7', 'Agr_Br_8', 'Agr_Br_9',
       'Chan_Br_1', 'Chan_Br_10', 'Chan_Br_11', 'Chan_Br_12', 'Chan_Br_2',
       'Chan_Br_3', 'Chan_Br_4', 'Chan_Br_5', 'Chan_Br_6', 'Chan_Br_7',
       'Chan_Br_8', 'Chan_Br_9', 'Gur_Br_1', 'Gur_Br_10', 'Gur_Br_11',
       'Gur_Br_12', 'Gur_Br_2', 'Gur_Br_3', 'Gur_Br_4', 'Gur_Br_5',
       'Gur_Br_6', 'Gur_Br_7', 'Gur_Br_8', 'Gur_Br_9']
    
    df_receipt_train.index = ['Agr_Br_1', 'Agr_Br_10', 'Agr_Br_2', 'Agr_Br_3', 'Agr_Br_4',
       'Agr_Br_5', 'Agr_Br_6', 'Agr_Br_7', 'Agr_Br_8', 'Agr_Br_9',
       'Chan_Br_1', 'Chan_Br_10', 'Chan_Br_11', 'Chan_Br_12', 'Chan_Br_2',
       'Chan_Br_3', 'Chan_Br_4', 'Chan_Br_5', 'Chan_Br_6', 'Chan_Br_7',
       'Chan_Br_8', 'Chan_Br_9', 'Gur_Br_1', 'Gur_Br_10', 'Gur_Br_11',
       'Gur_Br_12', 'Gur_Br_2', 'Gur_Br_3', 'Gur_Br_4', 'Gur_Br_5',
       'Gur_Br_6', 'Gur_Br_7', 'Gur_Br_8', 'Gur_Br_9']
    
    df_receipt_test.index = ['Agr_Br_1', 'Agr_Br_10', 'Agr_Br_2', 'Agr_Br_3', 'Agr_Br_4',
       'Agr_Br_5', 'Agr_Br_6', 'Agr_Br_7', 'Agr_Br_8', 'Agr_Br_9',
       'Chan_Br_1', 'Chan_Br_10', 'Chan_Br_11', 'Chan_Br_12', 'Chan_Br_2',
       'Chan_Br_3', 'Chan_Br_4', 'Chan_Br_5', 'Chan_Br_6', 'Chan_Br_7',
       'Chan_Br_8', 'Chan_Br_9', 'Gur_Br_1', 'Gur_Br_10', 'Gur_Br_11',
       'Gur_Br_12', 'Gur_Br_2', 'Gur_Br_3', 'Gur_Br_4', 'Gur_Br_5',
       'Gur_Br_6', 'Gur_Br_7', 'Gur_Br_8', 'Gur_Br_9']
    
#     df_pay_train.to_csv("df_pay_train.csv")
#     df_pay_test.to_csv("df_pay_test.csv")
#     df_receipt_train.to_csv("df_receipt_train.csv")
#     df_receipt_test.to_csv("df_receipt_test.csv")
    return df_pay_train, df_pay_test, df_receipt_train, df_receipt_test

In [5]:
st = time.time()
whole(datafr1)
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

C:\Users\RAKTIMA\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\RAKTIMA\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\RAKTIMA\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\RAKTIMA\anaco

C:\Users\RAKTIMA\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\RAKTIMA\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\RAKTIMA\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\RAKTIMA\anaco

Execution time: 779.2166519165039 seconds


In [6]:
df_pay_train

rmse        mape    avg_act   avg_pred      stdev        min  \
Agr_Br_1     9.729532   53.289070  26.605609  26.591775   4.861815  16.874036   
Agr_Br_10    7.871425   48.848111  19.484763  19.466421   4.588090   1.895625   
Agr_Br_2    15.197211   20.697658  59.052178  59.029115   8.622271  45.032661   
Agr_Br_3     7.700161   34.065623  20.819141  20.824162   2.293819  15.484944   
Agr_Br_4     5.260501   52.994245  11.984989  11.998308   1.637663   7.310370   
Agr_Br_5    12.294968   18.142912  54.819693  54.827531   3.997228  47.744273   
Agr_Br_6    17.612391   79.487704  17.036351  17.015242   5.231235   4.879835   
Agr_Br_7     8.694586   25.583542  28.637814  28.648590   4.612063  15.189564   
Agr_Br_8    12.861265  111.623957  55.577921  55.623923   6.741485  27.604599   
Agr_Br_9     9.008059   23.123915  35.216534  35.222628   4.526001  27.429462   
Chan_Br_1    6.452214   14.042403  34.669441  34.670736   3.227961  29.158331   
Chan_Br_10  11.008640   48.647034  25.586818  25.596138   3.446255  17.846286   
Chan_Br_11  10.001260   63.516310  14.708148  14.703443   2.524402   8.253774   
Chan_Br_12   5.942423   31.289378  17.745552  17.751045   2.760848  11.346138   
Chan_Br_2    9.471317   25.234547  31.264431  31.273917   7.490319  17.978929   
Chan_Br_3   25.953229   52.707492  50.344226  50.349281   8.339174  28.017091   
Chan_Br_4    9.709767   55.891796  16.551605  16.558819   4.206957   8.775270   
Chan_Br_5   10.473387   47.949514  21.176995  21.164336   3.277297  14.930613   
Chan_Br_6    9.939255   17.842957  46.530879  46.510529   4.283841  41.280294   
Chan_Br_7    4.791455   46.014310  10.709705  10.705181   2.004760   6.196833   
Chan_Br_8   12.500867   20.557862  53.241845  53.235516  12.084741  34.001816   
Chan_Br_9   12.664725   42.113570  36.946802  36.933416   5.227211  26.827662   
Gur_Br_1    16.950504   87.025708  42.740487  42.773077   9.709608  26.642086   
Gur_Br_10   26.373060   29.516001  79.844979  79.882373   5.984452  70.566456   
Gur_Br_11   23.410323   49.155565  43.428341  43.370524   9.960557  20.617450   
Gur_Br_12   16.337063  133.616146  25.700637  25.721621   5.502198  16.924956   
Gur_Br_2     6.209254   50.219317  15.926886  15.926703   2.300153  10.098144   
Gur_Br_3     3.161133   89.962148   4.886588   4.883685   0.923152   2.937549   
Gur_Br_4     5.094976   40.591464  12.399682  12.399048   2.106549   7.344676   
Gur_Br_5     5.679123   51.907357  12.049848  12.056284   2.239060   6.457106   
Gur_Br_6     7.343450   53.414723  13.795278  13.784297   2.216275   8.087101   
Gur_Br_7    21.056337   70.762928  65.828614  65.834771   8.996168  45.454358   
Gur_Br_8     8.817809   29.298091  27.309529  27.330060   4.887040  20.020425   
Gur_Br_9     9.083217   58.330129  20.205473  20.202910   3.861439  10.510432   

                  max  cnt_neg_pred  
Agr_Br_1    39.220637           0.0  
Agr_Br_10   32.632338           0.0  
Agr_Br_2    80.204340           0.0  
Agr_Br_3    28.177406           0.0  
Agr_Br_4    16.994037           0.0  
Agr_Br_5    63.016522           0.0  
Agr_Br_6    32.843529           0.0  
Agr_Br_7    40.098669           0.0  
Agr_Br_8    74.722223           0.0  
Agr_Br_9    48.682645           0.0  
Chan_Br_1   46.044444           0.0  
Chan_Br_10  35.508835           0.0  
Chan_Br_11  25.152657           0.0  
Chan_Br_12  28.134228           0.0  
Chan_Br_2   45.736374           0.0  
Chan_Br_3   85.067609           0.0  
Chan_Br_4   35.128555           0.0  
Chan_Br_5   29.917089           0.0  
Chan_Br_6   59.303756           0.0  
Chan_Br_7   18.370468           0.0  
Chan_Br_8   90.701808           0.0  
Chan_Br_9   54.633432           0.0  
Gur_Br_1    72.622216           0.0  
Gur_Br_10   99.139670           0.0  
Gur_Br_11   75.094453           0.0  
Gur_Br_12   43.653329           0.0  
Gur_Br_2    22.777825           0.0  
Gur_Br_3     7.752074           0.0  
Gur_Br_4    19.183127           0.0  
Gur_Br_5    18.875120           0.0  
Gur

In [7]:
df_pay_test

rmse        mape    avg_act   avg_pred     stdev        min  \
Agr_Br_1     6.488803   24.740171  18.825758  18.371711  2.443485  16.117141   
Agr_Br_10    8.109185   27.503305  23.272004  23.144309  3.104775  17.094986   
Agr_Br_2    21.280445   48.671012  48.875904  66.205057  7.115225  58.017401   
Agr_Br_3     8.566104   35.348265  19.309315  19.931948  1.452957  15.460889   
Agr_Br_4     5.833071   65.369997  10.994070  13.531617  0.918526  11.639754   
Agr_Br_5    13.326675   24.761806  49.601462  54.165640  3.803011  48.352425   
Agr_Br_6     7.240600   43.841841  11.578062  10.298210  3.959819   4.212471   
Agr_Br_7     8.985978   20.688656  27.165010  21.789923  3.131816  14.538728   
Agr_Br_8    24.826434   18.908219  74.911785  61.795369  4.285762  55.855928   
Agr_Br_9     8.790024   20.401381  34.487627  34.477099  3.161177  30.457641   
Chan_Br_1   10.358520   17.805244  36.117237  37.829377  2.187365  34.996612   
Chan_Br_10  13.561864   73.483623  23.372999  30.330535  2.677771  24.889108   
Chan_Br_11   9.347243   98.055795  13.772103  16.870750  1.799421  13.787580   
Chan_Br_12   5.592821   22.063788  20.212083  19.924827  1.581272  16.353423   
Chan_Br_2    9.862677   30.795682  33.315889  34.447952  2.475985  29.504131   
Chan_Br_3   22.374181   80.397500  32.127188  51.584336  5.646996  41.928675   
Chan_Br_4   15.209143  163.936107  14.262504  27.553236  2.757604  22.054708   
Chan_Br_5    7.774162   48.232770  17.715762  21.288932  2.813001  16.665259   
Chan_Br_6   11.962837   23.766092  41.310537  44.153614  3.644081  41.142394   
Chan_Br_7    4.213327   53.776025   8.329582  10.490099  1.526878   7.595895   
Chan_Br_8   29.130348   64.314897  49.730786  76.097752  4.305408  67.938049   
Chan_Br_9   17.720222   69.008839  29.520698  38.558360  4.638995  31.376669   
Gur_Br_1    12.257462   28.017838  44.360392  48.879478  8.549229  41.144588   
Gur_Br_10   36.954160   63.473227  66.517880  87.700030  4.854591  80.932101   
Gur_Br_11   24.582522   79.501523  32.387426  55.650983  7.067550  41.719419   
Gur_Br_12   22.442901  132.541572  19.792953  41.150686  1.998914  37.390854   
Gur_Br_2     5.400169   41.078044  14.269098  17.257260  1.727338  13.663232   
Gur_Br_3     3.244854  148.339168   3.596902   5.807460  0.712281   4.529511   
Gur_Br_4     3.387958   30.599719  11.962196  12.860155  1.696924  10.065651   
Gur_Br_5     6.954427   59.944565  13.357550  13.024751  1.849598   9.012573   
Gur_Br_6     8.197108   94.752766  10.477106  17.068396  1.763490  13.066542   
Gur_Br_7    28.237474   61.026919  52.949077  73.588553  8.333948  57.656920   
Gur_Br_8     9.393881   42.196639  23.641840  26.599795  4.637647  21.572483   
Gur_Br_9     6.775521   30.445744  21.307733  22.994663  3.394042  16.553627   

                  max  cnt_neg_pred  
Agr_Br_1    25.217866           0.0  
Agr_Br_10   31.036353           0.0  
Agr_Br_2    80.292824           0.0  
Agr_Br_3    21.112903           0.0  
Agr_Br_4    14.882426           0.0  
Agr_Br_5    61.360197           0.0  
Agr_Br_6    17.667949           0.0  
Agr_Br_7    27.244525           0.0  
Agr_Br_8    69.778945           0.0  
Agr_Br_9    42.767877           0.0  
Chan_Br_1   42.300682           0.0  
Chan_Br_10  35.761365           0.0  
Chan_Br_11  19.757338           0.0  
Chan_Br_12  22.984193           0.0  
Chan_Br_2   39.491508           0.0  
Chan_Br_3   60.403389           0.0  
Chan_Br_4   34.682501           0.0  
Chan_Br_5   27.011425           0.0  
Chan_Br_6   53.591928           0.0  
Chan_Br_7   13.273956           0.0  
Chan_Br_8   85.219038           0.0  
Chan_Br_9   48.458236           0.0  
Gur_Br_1    70.612564           0.0  
Gur_Br_10   99.177706           0.0  
Gur_Br_11   70.889212           0.0  
Gur_Br_12   44.868396           0.0  
Gur_Br_2    21.568506           0.0  
Gur_Br_3     6.704540           0.0  
Gur_Br_4    16.600580           0.0  
Gur_Br_5    16.860448           0.0  
Gur_Br_6    20.277248           0.0  


In [8]:
df_receipt_train

rmse        mape    avg_act   avg_pred      stdev  \
Agr_Br_1    109.554428   77.748186  73.489336  73.526326  68.542288   
Agr_Br_10    24.481246   46.819657  27.740056  27.737678  11.021410   
Agr_Br_2     13.277734   22.064929  51.321985  51.322343   8.509828   
Agr_Br_3      9.088665   24.985175  31.913836  31.917645   5.155305   
Agr_Br_4      4.280635   58.806046   8.595349   8.600643   1.811039   
Agr_Br_5     15.642436   32.113479  45.930863  45.933194   4.592924   
Agr_Br_6     13.739750   30.823468  25.542850  25.552814   7.032984   
Agr_Br_7      7.390939   49.624987  15.880193  15.872251   1.740435   
Agr_Br_8     17.194231   41.527617  65.153195  65.182930  12.622137   
Agr_Br_9     10.631821   57.351779  24.310090  24.315674   5.530563   
Chan_Br_1     7.896624   18.210368  34.230051  34.228602   3.730478   
Chan_Br_10    8.057669   66.568512  18.970062  18.965524   2.694298   
Chan_Br_11    6.795253   68.200190   9.814000   9.804118   2.299672   
Chan_Br_12    6.104091   30.348582  17.411817  17.411899   3.848898   
Chan_Br_2     9.841975   25.650549  29.686727  29.675569   7.370125   
Chan_Br_3    10.708579   30.924527  32.344388  32.341235   4.530740   
Chan_Br_4     8.160264  136.647338  10.364703  10.358975   3.079517   
Chan_Br_5     6.480890   47.242434  14.389329  14.390583   3.171210   
Chan_Br_6    16.595879   33.797619  61.755956  61.788670   7.944432   
Chan_Br_7     8.275646   61.114827  16.082519  16.064870   5.380468   
Chan_Br_8    11.306645   21.957876  43.854434  43.859056   8.912890   
Chan_Br_9    11.885748   31.795737  41.389430  41.382620   6.447121   
Gur_Br_1      9.341691   41.827636  24.042401  24.027891   6.022778   
Gur_Br_10    14.894203   25.849198  53.235997  53.212086   7.271228   
Gur_Br_11    51.694137   44.180924  35.363798  35.298985   9.565310   
Gur_Br_12    12.122263   84.911990  16.434816  16.421756   4.785638   
Gur_Br_2      5.443355   40.809063  15.045334  15.037945   3.012411   
Gur_Br_3      3.538490   63.871430   6.490022   6.486111   1.519096   
Gur_Br_4      5.630410   38.378187  14.548416  14.550600   5.067818   
Gur_Br_5      7.634634   68.096345  10.517415  10.505697   2.667389   
Gur_Br_6      5.258615   84.881847   9.146587   9.149974   1.693775   
Gur_Br_7     19.974577   69.155965  71.637812  71.658685  16.536314   
Gur_Br_8      9.115573   34.535763  25.080497  25.084670   4.587577   
Gur_Br_9      5.768914   64.352811  11.486880  11.480462   2.598013   

                  min         max  cnt_neg_pred  
Agr_Br_1    -2.294038  295.620449           2.0  
Agr_Br_10    1.768958   75.750668           0.0  
Agr_Br_2    36.808469   78.975498           0.0  
Agr_Br_3    22.911835   47.797742           0.0  
Agr_Br_4     4.360834   12.369356           0.0  
Agr_Br_5    32.586763   54.700696           0.0  
Agr_Br_6    15.365406   47.098840           0.0  
Agr_Br_7    10.585174   21.808303           0.0  
Agr_Br_8    26.372500  104.620834           0.0  
Agr_Br_9    13.025700   42.728572           0.0  
Chan_Br_1   26.857307   47.876086           0.0  
Chan_Br_10  13.318642   28.434555           0.0  
Chan_Br_11   4.764759   17.699981           0.0  
Chan_Br_12   9.307778   31.230314           0.0  
Chan_Br_2   17.341952   46.703220           0.0  
Chan_Br_3   25.793420   44.661923           0.0  
Chan_Br_4    4.738534   23.813664           0.0  
Chan_Br_5    8.680921   24.865412           0.0  
Chan_Br_6   43.349491   92.661224           0.0  
Chan_Br_7    9.987198   32.019127           0.0  
Chan_Br_8   28.665922   74.403208           0.0  
Chan_Br_9   28.053345   65.444900           0.0  
Gur_Br_1    14.537514   43.014079           0.0  
Gur_Br_10   34.522380   78.226233           0.0  
Gur_Br_11   20.007888   64.655763           0.0  
Gur_Br_12    7.024728   32.055501           0.0  
Gur_Br_2     8.915548   23.489503           0.0  
Gur_Br_3     3.650210   13.474497           0.0  
Gur_Br_4     4.129747   29.269477           0.0  
Gur_Br_5     6.072902   18.79194

In [9]:
df_receipt_test

rmse        mape    avg_act    avg_pred      stdev  \
Agr_Br_1    273.694051  758.374059  81.518766  324.582947  19.792460   
Agr_Br_10    13.833722   29.937776  28.327625   19.912194   4.933377   
Agr_Br_2     14.226193   22.061773  57.216395   59.789485   5.587772   
Agr_Br_3      8.708962   30.155039  27.075931   34.123623   3.782930   
Agr_Br_4      6.806693   39.077242  11.651329   11.227912   0.905457   
Agr_Br_5     12.303758   25.673421  43.438392   46.471433   1.906136   
Agr_Br_6     11.361512   57.398135  22.475643   32.105271   5.655575   
Agr_Br_7     12.711212   37.419754  23.082437   14.371490   1.389545   
Agr_Br_8     19.903879   24.965789  74.846870   82.561985   8.616951   
Agr_Br_9     10.766292   44.117635  27.054033   33.606564   3.717847   
Chan_Br_1    12.148190   16.110023  37.121642   37.538594   2.847983   
Chan_Br_10    8.198004   82.655070  13.565667   19.565101   2.335541   
Chan_Br_11    8.017485  108.611939  10.098322   13.395813   1.742729   
Chan_Br_12    7.630119   61.993076  16.646748   20.671167   2.860029   
Chan_Br_2     8.950959   23.995757  32.564767   32.498379   2.721856   
Chan_Br_3     9.973879   39.634859  26.711426   31.888228   3.986750   
Chan_Br_4     9.385474  123.001306  11.699937   18.381819   1.939370   
Chan_Br_5     5.332774   33.654261  12.738451   12.658636   2.797885   
Chan_Br_6    13.301347   19.932426  54.305320   56.876778   6.449953   
Chan_Br_7     7.050932   34.014649  16.242790   14.628914   4.769529   
Chan_Br_8    20.724825   46.072677  47.975335   61.252341   3.617179   
Chan_Br_9    10.390235   25.643612  40.705174   42.540009   5.419204   
Gur_Br_1      7.031770   32.273957  23.919014   25.576204   5.746836   
Gur_Br_10    19.429997   39.470390  52.216141   60.274306   4.638810   
Gur_Br_11    12.578214   28.938924  37.860382   42.579142   8.318352   
Gur_Br_12    15.801455  127.431384  19.180617   32.559096   2.260303   
Gur_Br_2      6.235141   44.770058  14.051305   16.959081   2.106837   
Gur_Br_3      2.665037   43.201931   6.657001    7.708445   1.088246   
Gur_Br_4      8.237242   63.985332  12.789949   16.642602   4.462701   
Gur_Br_5      5.776506  152.945907   6.322466   10.421955   1.771697   
Gur_Br_6      4.839751   79.580226   7.890826   11.646714   1.313743   
Gur_Br_7     20.766893   36.892553  52.306852   66.268900  15.885876   
Gur_Br_8      9.224741   40.148756  21.540116   26.020258   4.568851   
Gur_Br_9      6.414750   52.031046  14.509505   16.081719   2.284139   

                   min         max  cnt_neg_pred  
Agr_Br_1    290.112115  363.029740           0.0  
Agr_Br_10    12.607165   30.843187           0.0  
Agr_Br_2     51.871936   74.701036           0.0  
Agr_Br_3     29.418153   44.326462           0.0  
Agr_Br_4      9.387307   12.653578           0.0  
Agr_Br_5     42.829550   50.950163           0.0  
Agr_Br_6     25.634482   48.089484           0.0  
Agr_Br_7     10.450911   16.357972           0.0  
Agr_Br_8     68.680167  102.608455           0.0  
Agr_Br_9     27.281967   44.080454           0.0  
Chan_Br_1    32.139786   43.135655           0.0  
Chan_Br_10   14.802115   23.798387           0.0  
Chan_Br_11   10.308285   17.133597           0.0  
Chan_Br_12   15.670987   27.826246           0.0  
Chan_Br_2    27.796432   40.914479           0.0  
Chan_Br_3    27.079668   43.016305           0.0  
Chan_Br_4    15.016142   23.498965           0.0  
Chan_Br_5     8.557988   19.362455           0.0  
Chan_Br_6    42.980006   71.111829           0.0  
Chan_Br_7     9.954738   28.747717           0.0  
Chan_Br_8    52.530689   69.280108           0.0  
Chan_Br_9    32.451916   55.200263           0.0  
Gur_Br_1     19.353733   41.202433           0.0  
Gur_Br_10    49.612292   68.421796           0.0  
Gur_Br_11    33.497097   62.916203           0.0  
Gur_Br_12    26.301913   35.728914           0.0  
Gur_Br_2     13.740667   22.377286           0.0  
Gur_Br_3      6.355585    9.912901           0.0  
Gur_Br_4     11.13